In [9]:
# Diego Carrillo Torres A01612532
from mesa import Agent, Model 
from mesa.time import SimultaneousActivation # para activar a todos los agentes de manera simultanea.
from mesa.datacollection import DataCollector # para obtener el grid completo cada paso usarlo para graficarlo.

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np # paquetes para manejar valores númericos.
import pandas as pd

import time # paquetes para medir el tiempo de ejecución del algoritmo.
import datetime

NUM_MOV = 0

def get_grid(model): # función auxiliar para guardar el grid de cada uno de los agentes.
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
            if isinstance(content, Vacuum):
                grid[x][y] = 2
            elif isinstance(content, Floor):
                if content.live == 1:
                    grid[x][y] = 1
                else:
                    grid[x][y] = 0
            else:
                grid[x][y] = 0
    return grid

In [10]:
class RandomWalker(Agent):
    grid = None
    x = None
    y = None
    moore = True

    def __init__(self, unique_id, pos, model, moore=True):
        super().__init__(unique_id, model)
        self.pos = pos
        self.moore = moore

    def random_move(self):
        next_moves = self.model.grid.get_neighborhood(self.pos, self.moore, True)
        next_move = self.random.choice(next_moves)
        self.model.grid.move_agent(self, next_move)

In [11]:
class Vacuum(RandomWalker):
    def __init__(self, unique_id, pos, model, moore):
        super().__init__(unique_id, pos, model, moore=moore)
        self.live=2

    def step(self):
        self.random_move()
        global NUM_MOV
        NUM_MOV = NUM_MOV + 1
        # Si esta sucio, limpia
        x, y = self.pos
        this_cell = self.model.grid.get_cell_list_contents([self.pos])
        dirt = [obj for obj in this_cell if isinstance(obj, Floor)]
        if len(dirt) > 0:
            dirt_to_eat = self.random.choice(dirt)
            # Elimina suciedad
            dirt_to_eat.live = 0
                
class Floor(Agent): # Para representar a un agente o celda con estado 1 o 0
    def __init__(self, unique_id, pos, model): # Crea un agente con estado inicial aleatorio de 0 o 1 y le asigna un identificador 
        super().__init__(unique_id, model)
        self.live = 1
        self.pos=pos
        self.next_state = None
    
    def step(self):
        this_cell = self.model.grid.get_cell_list_contents([self.pos])

In [12]:
from mesa import Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector


class CleaningModel(Model):
    def __init__(self, width, height, initial_dirt, initial_bot):
        self.initial_dirt=initial_dirt
        self.initial_bot=initial_bot
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
        
        for i in range(self.initial_dirt): # generar piso sucio
            x = self.random.randrange(width)
            y = self.random.randrange(height)
            dirt = Floor(i,(x, y), self)
            self.grid.place_agent(dirt, (x, y))
            self.schedule.add(dirt)

        for i in range(self.initial_bot): # generar bots aspiradoras
            bot = Vacuum(i+initial_dirt, (1, 1), self,True)
            self.grid.place_agent(bot, (1, 1))
            self.schedule.add(bot)
        
        self.datacollector = DataCollector( # se obtiene el grid completo usando collector
            model_reporters={"Grid": get_grid})
    def isallclean(self):
        filthy_cells = 0
        for cell in model.grid.coord_iter():
            cell_content, x, y = cell
            for content in cell_content:
                if isinstance(content, Floor):
                    if content.live == 1:
                        filthy_cells = filthy_cells + 1
                
        if filthy_cells>=1:
            return False
        else:
            return True
        
    def step(self): # En cada paso el colector toma la información  y la almacena en el grid para luego graficarlo.
        self.schedule.step()
        self.datacollector.collect(self)

In [13]:
# Parámetros Iniciales
TIEMPO_MAXIMO_EJECUCION = 0.005 #Tiempo Maximo de Ejecucion
initial_bots            = 5 #Numero de Agentes
initial_dirt_percentage = .16 #Porcentaje celdas sucias
width                   = 6 #Tamaño de M
height                  = 6 #Tamaño de N

#initial_bots            = int(input('Numero inicial de agentes: '))
#initial_dirt_percentage = float(input('Porcentaje inicial de celdas sucias: '))
#width                   = int(input('Width M: '))
#height                  = int(input('Height N: '))

initial_dirt = round(initial_dirt_percentage*(width*height))

start_time = time.time()
tiempo_inicio=str(datetime.timedelta(seconds=TIEMPO_MAXIMO_EJECUCION))
model = CleaningModel(width, height, initial_dirt, initial_bots)
while((time.time()-start_time) < TIEMPO_MAXIMO_EJECUCION and model.isallclean() == False):
    model.step()
tiempo = str(datetime.timedelta(seconds=(time.time() - start_time))) # tiempo que le tomó correr al modelo

all_grid = model.datacollector.get_model_vars_dataframe()
#print(type(all_grid))
#print(all_grid)

In [14]:
%%capture
fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='Greys')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [15]:
anim

# Información recopilada de la ejecución

In [16]:
last_grid = all_grid.iloc[-1][0].astype(int)
#print(last_grid)
clean_cells=0

clean_cells = clean_cells + np.count_nonzero(last_grid == 2)
clean_cells = clean_cells + np.count_nonzero(last_grid == 0)
print('Celdas limpias: ', clean_cells)

print('Tiempo: ', tiempo)
porcentajeCeldasLimpias = (clean_cells*100)/(width*height);
print('Porcentaje de celdas limpias: ', porcentajeCeldasLimpias, '%')
print('Número de movimientos realizados por los agentes: ', NUM_MOV)

Celdas limpias:  36
Tiempo:  0:00:00.001994
Porcentaje de celdas limpias:  100.0 %
Número de movimientos realizados por los agentes:  55
